In [ ]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, hidden_size, layers):
        super(LSTM,self).__init__()
        self.lstm = nn.LSTM(input_size=3, hidden_size=hidden_size,num_layers=layers,batch_first=True,bidirectional=True)
        self.fc=nn.Linear(hidden_size*2,6)
        self.hidden_size = hidden_size
        self.layers = layers

    def forward(self,x):
        x,_ = self.lstm(x)
        x=x[:,-1,:]
        x = self.fc(x)
        return x

In [ ]:
model=LSTM(128, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset

class LorenzSlidingWindowDataset(Dataset):
    def __init__(self, folder_path, window_size, stride_length, index_map=None, file_info=None):
        self.folder_path = folder_path
        self.window_size = window_size
        self.stride_length = stride_length

        self.file_info = file_info if file_info is not None else self._scan_files()
        self.index_map = index_map if index_map is not None else self._build_index_map()

        if self.file_info is not None:
            self.data_frames = [
                pd.read_csv(os.path.join(self.folder_path, info['filename']))
                for info in self.file_info
            ]
            
    def _scan_files(self):
        files = os.listdir(self.folder_path)
        info = []
        for f in files:
            parts = f.split('_')
            index = parts[0]
            target = parts[1:]
            info.append({'filename': f, 'index': index, 'target': target})
        return info

    def _build_index_map(self):
        index_map = []
        fixed_length = 10000
        max_start = fixed_length - self.window_size
    
        for file_idx in range(len(self.file_info)):
            for i in range(0, max_start + 1, self.stride_length):
                index_map.append((file_idx, i))
    
        return index_map


    def __len__(self):
        return len(self.index_map)

    def __getitem__(self, idx):
        file_idx, row_start = self.index_map[idx]
        file_meta = self.file_info[file_idx]
        file_path = file_meta['filename']

        df = self.data_frames[file_idx]
        x = df.iloc[:self.window_size, 0:3].values 
        y = [float(i) for i in file_path[:-4].split("_")[1:]]

        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [ ]:
from torch.utils.data import DataLoader

full_dataset = LorenzSlidingWindowDataset(
    folder_path="/kaggle/input/lorenz-data1/lorenz_data_separate/",
    window_size=50,
    stride_length=25
)

from sklearn.model_selection import train_test_split
train_indices, val_indices = train_test_split(
    full_dataset.index_map, test_size=0.2, random_state=42
)

train_dataset = LorenzSlidingWindowDataset(
    folder_path="/kaggle/input/lorenz-data1/lorenz_data_separate/",
    window_size=50,
    stride_length=25,
    index_map=train_indices,
    file_info=full_dataset.file_info
)

val_dataset = LorenzSlidingWindowDataset(
    folder_path="/kaggle/input/lorenz-data1/lorenz_data_separate/",
    window_size=50,
    stride_length=25,
    index_map=val_indices,
    file_info=full_dataset.file_info
)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2048, shuffle=False)

In [ ]:
model.load_state_dict(torch.load("/kaggle/input/chaos-lstm/pytorch/92/1/92.pth", weights_only=True))

In [ ]:
from tqdm import tqdm
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
epochs = 20

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [TRAIN]")
    for x_batch, y_batch in train_bar:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        preds = model(x_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x_batch.size(0)
        train_bar.set_postfix(loss=loss.item())

    train_loss /= len(train_loader.dataset)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [VAL]")
        for x_batch, y_batch in val_bar:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            preds = model(x_batch)
            loss = criterion(preds, y_batch)

            val_loss += loss.item() * x_batch.size(0)
            val_bar.set_postfix(loss=loss.item())

    val_loss /= len(val_loader.dataset)

    print(f"\nEpoch {epoch+1} Summary: Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}\n")


In [ ]:
torch.save(model.state_dict(), "92.pth")

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_targets = []
    total_loss = 0.0
    criterion = nn.MSELoss()

    with torch.no_grad():
        for x_batch, y_batch in tqdm(dataloader, desc="Evaluating"):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            preds = model(x_batch)
            loss = criterion(preds, y_batch)
            total_loss += loss.item() * x_batch.size(0)

            all_preds.append(preds.cpu())
            all_targets.append(y_batch.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_targets = torch.cat(all_targets).numpy()

    mse = mean_squared_error(all_targets, all_preds)
    mae = mean_absolute_error(all_targets, all_preds)
    avg_loss = total_loss / len(dataloader.dataset)

    mse_per_target = ((all_preds - all_targets) ** 2).mean(axis=0)
    mae_per_target = np.abs(all_preds - all_targets).mean(axis=0)

    print(f"\nEvaluation Results")
    print(f"Overall MSE: {mse:.4f}, MAE: {mae:.4f}, Avg Loss: {avg_loss:.4f}")
    print("\n📊 Per-Target MSE / MAE:")
    for i in range(all_preds.shape[1]):
        print(f"Target {i+1}: MSE = {mse_per_target[i]:.4f}, MAE = {mae_per_target[i]:.4f}")

    return all_preds, all_targets, mse_per_target, mae_per_target

In [ ]:
from tqdm import tqdm
model = model.to("cuda")
val_preds, val_targets, mses, maes = evaluate_model(model, val_loader, "cuda")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_predictions(preds, targets, num_targets=6, sample_size=200):
    """
    Plots predicted vs true values for each target.

    Parameters:
        preds (np.ndarray): Model predictions (N x num_targets)
        targets (np.ndarray): True labels (N x num_targets)
        num_targets (int): Number of target variables
        sample_size (int): Number of samples to plot per target
    """
    assert preds.shape == targets.shape, "Predictions and targets must have the same shape."
    sample_size = min(sample_size, preds.shape[0])

    plt.figure(figsize=(18, 12))
    
    for i in range(num_targets):
        plt.subplot(2, 3, i + 1)
        
        # Scatter plot
        plt.scatter(
            targets[:sample_size, i],
            preds[:sample_size, i],
            alpha=0.6,
            s=10,
            c='tab:blue',
            label='Prediction'
        )
        
        min_val = min(targets[:sample_size, i].min(), preds[:sample_size, i].min())
        max_val = max(targets[:sample_size, i].max(), preds[:sample_size, i].max())
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='Ideal')

        plt.xlabel("True")
        plt.ylabel("Predicted")
        plt.title(f"Target {i+1}")
        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.suptitle("Predicted vs True Values (First 200 Samples)", fontsize=18, y=1.02)
    plt.show()

In [ ]:
plot_predictions(val_preds, val_targets)